In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q tensorflow-cpu
!pip install -q -U keras-nlp tensorflow-hub
!pip install -q -U keras>=3
!pip install -U tensorflow-text

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.3.2 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow-cpu 2.16.1 requires keras>=3.0.0, but you have keras 2.15.0 which is incompatible.
tensorflow-cpu 2.16.1 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.

In [1]:
import os

# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 100% of TPU memory to minimize memory fragmentation
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

import keras
import keras_nlp

# for reproducibility
keras.utils.set_random_seed(42)

2024-04-27 07:56:19.496897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 07:56:19.496997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 07:56:19.763962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_2b_en")


Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [3]:
# formatting utility
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [61]:
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines of the Gemma family of models.
  documented here: https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message + self.__END_TURN__)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    prompt = message
    response = self.model.generate(prompt, max_length=1024)
    result = response.replace(prompt, "")  # Extract only the new response
    
    return result

In [62]:
import pandas as pd
vis = pd.read_csv('/kaggle/input/visitor/visitors.csv')
exh = pd.read_csv('/kaggle/input/exhibitor/exhibitors (1).csv')

In [63]:
vis = vis[vis['profession'] != 'other']

vis.drop_duplicates(subset='email', keep='first', inplace=True)

In [64]:
pd.set_option('display.max_columns', None)

In [75]:
from tqdm import tqdm
c =0 

chat = ChatState(gemma_lm)

for idx, row in tqdm(vis.head(1000).iterrows(), total=1000):
    
    if c >= 500:
        if row['profession'] != 'other':
            message = f'''can you generate an about me for a person named {row['company_repName']} who
              works as an {row['profession']} at {row['company_name']}. Make sure what
              you generate tells me about the person's interests in technology.
              respond in a maximum of 32 words'''
        else:
            message = f'''can you generate an about me for a person named {row['company_repName']} who
              works at {row['company_name']}. Make sure what
              you generate tells me about the person's interests in technology.
              respond in a maximum of 32 words'''
        res = chat.send_message(message)
    
        list.append(res)
        
    c +=1
    
print(list)

100%|██████████| 1000/1000 [26:07<00:00,  1.57s/it]

['.\n\nDriven by innovation, I leverage technology to solve complex business challenges. My passion lies in exploring cutting-edge technologies and implementing them to drive efficiency and growth.', '.\n\nDriven procurement professional with 5+ years of experience in boiler manufacturing industry. Passionate about leveraging technology to optimize supply chain and enhance customer experience.', '.\n\nDriven by innovation, I leverage technology to optimize processes and enhance efficiency at Utopian sugars Ltd. My passion lies in exploring cutting-edge technologies that drive automation and predictive maintenance.', '.\n\nKishor Kumbhar is a passionate Renewable Energy Project Developer with a keen interest in leveraging technology to drive sustainable energy solutions. He thrives in fast-paced environments, driving project development and implementation.', '.\n\nDriven by innovation, I leverage technology to optimize purchasing processes at Zeel Engineering and Fabrication. Passionate

In [66]:
print(list)

['.\n\nDriven by innovation, I leverage technology to solve complex business challenges. My passion lies in exploring cutting-edge technologies and implementing them to drive efficiency and growth.', '.\n\nDriven procurement professional with 5+ years of experience in boiler manufacturing industry. Passionate about leveraging technology to optimize supply chain and enhance customer experience.', '.\n\nDriven by innovation, I leverage technology to optimize processes and enhance efficiency at Utopian sugars Ltd. My passion lies in exploring cutting-edge technologies that drive automation and predictive maintenance.', '.\n\nKishor Kumbhar is a passionate Renewable Energy Project Developer with a keen interest in leveraging technology to drive sustainable energy solutions. He thrives in fast-paced environments, driving project development and implementation.', '.\n\nDriven by innovation, I leverage technology to optimize purchasing processes at Zeel Engineering and Fabrication. Passionate

In [102]:
list[521] 

'.\n\nDriven engineer with a passion for innovative technologies. I leverage my expertise in electrical and mechanical engineering to drive operational excellence and efficiency. Seeking opportunities to leverage my skills at Gharda Chemicals Limited, Panoli.'

In [103]:
list[501]

'.\n\nHitesh Mehta is a seasoned Purchase Personnel with over 10 years of experience in the boiler manufacturing industry. Passionate about optimizing supply chain and leveraging technology to drive efficiency.'

In [101]:
list[499]

'.\n\nDriven engineer with a passion for innovative technologies. I leverage my expertise in electrical engineering to optimize power generation systems. Seeking opportunities to leverage cutting-edge solutions at Maharashtra State Power Generation Co. Ltd. to enhance efficiency and sustainability.'

In [90]:
rd = pd.DataFrame(list)
vis['about_me'] = rd

In [83]:
list[1000]

'.\n\nDriven procurement professional with 5+ years of experience in boiler manufacturing industry. Passionate about leveraging technology to optimize supply chain and enhance efficiency.'

In [98]:
vis.head(500)

,id,company_repName,company_name,blood_group,mobile_no,email,designation,country,password,visitor,profession,role,createdAt,updatedAt,city,state,vaccine_certificate,id_certificate,company_logo,alternative_email,address_line1,address_line2,address_line3,company_repProfile,firebaseToken,socket_id,about_me,account_deletion_request,aboutme
0,3,Abhijit Alok,Kriti Trade Impex Pvt Ltd New Delhi,B+,9.195824e+11,abhijit.a@kimpexflow.com,Assistant Manager,India,$2a$10$6bVOOQKdY2/Bo1Vu0pp7Qez1qirp81GMFfwHvns...,NaN,Engineer/Operatorlookingfornewtechnology,visitor,2021-12-14 10:54:33,2021-12-14 10:54:33,Delhi,Delhi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,".\n\nDriven by innovation, I leverage technolo..."
1,4,SANJAY TORE,MTAR Technologies Ltd,O+,9.191775e+11,sanjay.tore@mtar.in,Business Development Manager,India,$2a$10$4K2KOgHWVCWxuWwoUB0hZuuAOL1RuX/oD1rSppN...,NaN,Purchase personnel from boiler manufacturing c...,visitor,2021-12-14 11:40:34,2022-08-22 09:30:56,Hyderabad,Telanagana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.\n\nDriven procurement professional with 5+ y...,0,.\n\nDriven procurement professional with 5+ y...
2,5,Sanjay Dnyanu Pawar,Utopian sugars Ltd,A+,9.183291e+11,sdpawar22@gmail.com,Boiler Attendant,India,$2a$10$Kcix6Sn0B2OT3sBdSiFOP.i5sbK3jKB/w/FhVqO...,NaN,Engineer/Operatorlookingfornewtechnology,visitor,2021-12-15 07:24:23,2021-12-15 07:24:23,Solapur,Maharashtra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,".\n\nDriven by innovation, I leverage technolo..."
3,6,Kishor Kumbhar,Berkeley Energy Commercial and Industrial Soli...,B+,9.180568e+11,kishor.kumbhar@be-cis.com,Assistant Manager - BD,India,$2a$10$Sr0tGU/i4T7ApyGz0Iy88Odjy.Gh8NPX3OFE.9n...,NaN,RenewableEnergyProjectDeveloper,visitor,2021-12-20 09:38:00,2022-08-30 09:39:01,Pune,Maharashtra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.\n\nKishor Kumbhar is a passionate Renewable ...,0,.\n\nKishor Kumbhar is a passionate Renewable ...
4,7,Hardik Ambaliya,Zeel Engineering and Fabrication,A+,9.197272e+11,zeel.fabrication@gmail.com,Founder,India,$2a$10$gmA1xn96l44vvfmA2Qbeke0dFC3JUPAQ2etjN60...,NaN,PurchasePersonnelFromBoilerManufacturingCompany,visitor,2021-12-20 17:46:55,2021-12-20 17:46:55,Surat,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,".\n\nDriven by innovation, I leverage technolo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,1063,Hirenbhai Parmar,Yogeshwar engineering works,NaN,9.199792e+11,hp1983.parmar@gmail.com,NaN,India,$2a$10$IWhu5jhNByiFx3642y8puuE7qUxv31KEuaXb8SR...,NaN,Engineer / operator looking for new technology,visitor,2022-08-04 10:33:30,2022-08-04 10:35:34,Jetpur,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by efficiency and innovation, I le...",0,NaN
1027,1064,Vivek Gupta,BAYLITE,NaN,9.194402e+11,baulite.office@gmail.com,NaN,NaN,$2a$10$xaPTyuHJipeGQV6rhGKv2ehQvT3EkxtdHAxG/hI...,NaN,Purchase personnel from boiler-user industry,visitor,2022-08-04 10:36:35,2022-08-04 10:36:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,NaN
1028,1065,Sundeep Vasan,Steam-Therm Consultancy,NaN,9.189995e+11,info@steam-therm.com,NaN,India,$2a$10$8mVuYVSguY9wWxoD6vnuGe4UhbX4waB56qxBa3q...,NaN,Design engineer / consultant,visitor,2022-08-04 10:43:13,2022-08-17 14:36:38,Dombivli,Maharashtra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.\n\nAniruddha Bhokare is a passionate procure...,0,NaN
1029,1066,Jayant Behl,VeeEss & Company,NaN,9.199713e+11,behl94@yahoo.com,NaN,NaN,$2a$10$g1WhMiZWj3M19f88w7V.teAwc54rnfNoFnunCYM...,NaN,Agencies dealing in instrumentation,visitor,2022-08-04 11:09:26,2022-08-04 11:09:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.\n\nMilind Shetkar is a passionate Purchase P...,0,NaN


In [104]:
vis.to_csv('visupd.csv')